In [1]:
%load_ext google.cloud.bigquery

import numpy as np
import pandas as pd

#### Fake dataset 1 - prosty graf

In [2]:
fake1 = pd.DataFrame([
     {'prev':'A', 'next':'B', 'prob':0.3}
    ,{'prev':'A', 'next':'C', 'prob':0.7}
    ,{'prev':'B', 'next':'D', 'prob':0.2}
    ,{'prev':'B', 'next':'E', 'prob':0.8}
    ,{'prev':'C', 'next':'E', 'prob':1.0}
    ,{'prev':'D', 'next':'N_CONV', 'prob':1.0}
    ,{'prev':'E', 'next':'N_CONV', 'prob':0.1}
    ,{'prev':'E', 'next':'CONV', 'prob':0.9}
])

print(fake1)

  prev    next  prob
0    A       B   0.3
1    A       C   0.7
2    B       D   0.2
3    B       E   0.8
4    C       E   1.0
5    D  N_CONV   1.0
6    E  N_CONV   0.1
7    E    CONV   0.9


In [64]:
class Crawler:
    _index = 0
    _pairs = pd.DataFrame()
    _paths = []
    
    def __init__(self, pairs = None, prev_history=['A']):
        Crawler._index += 1
        print(Crawler._index)
        self.index = Crawler._index
        if pairs is not None:
            Crawler._pairs = pairs
        self.history = prev_history
    
    def step(self):
        current = self.history[-1]
        exits = Crawler._pairs[Crawler._pairs['prev'] == current]['next']
        if exits.empty:
            Crawler._paths.append(self.history)
            print('Crawler {} finished'.format(self.index))
        else:
            count = 0
            for e in exits:
                print('Crawler {} takes next step'.format(self.index))
                if count == 0:
                    self.history.append(e)
                    self.step()
                else:
                    new = Crawler(prev_history = self.history)
                    print('Created new crawler {}'.format(new.index))
                    new.history.append(e)
                    new.step()
                count += 1
        
 
c1 = Crawler(pairs = fake1)
c1.step()


1
Crawler 1 takes next step
Crawler 1 takes next step
Crawler 1 takes next step
Crawler 1 finished
Crawler 1 takes next step
2
Created new crawler 2
Crawler 2 takes next step
Crawler 2 finished
Crawler 2 takes next step
3
Created new crawler 3
Crawler 3 finished
Crawler 1 takes next step
4
Created new crawler 4
Crawler 4 takes next step
Crawler 4 takes next step
Crawler 4 finished
Crawler 4 takes next step
5
Created new crawler 5
Crawler 5 finished


In [52]:
Crawler._paths

[['A', 'B', 'D', 'N_CONV', 'E', 'N_CONV', 'CONV', 'C', 'E', 'N_CONV', 'CONV'],
 ['A', 'B', 'D', 'N_CONV', 'E', 'N_CONV', 'CONV', 'C', 'E', 'N_CONV', 'CONV'],
 ['A', 'B', 'D', 'N_CONV', 'E', 'N_CONV', 'CONV', 'C', 'E', 'N_CONV', 'CONV'],
 ['A', 'B', 'D', 'N_CONV', 'E', 'N_CONV', 'CONV', 'C', 'E', 'N_CONV', 'CONV'],
 ['A', 'B', 'D', 'N_CONV', 'E', 'N_CONV', 'CONV', 'C', 'E', 'N_CONV', 'CONV']]